# Data Rate estimates for Station 196

## Import Libraries

In [ ]:
import pps_hitmaps
import ROOT
import Helper20240606
import pandas
from math import factorial
from math import exp

## Get Sensor

In [ ]:
new_sensor = pps_hitmaps.RectangularPadSensor(NumSmallerCols = 16)
new_sensor_ti = pps_hitmaps.RectangularPadSensor(NumSmallerCols = 16, PadSpacing=0.01)
sensor = new_sensor_ti
fig = new_sensor.preview()

## Load Hitmaps and check everything is ok

In [ ]:
from pathlib import Path

backgroundFlux = 5*10**12
backgroundTitle = "5E12 p/cm^2"
#noBackgroundFlux = 200  # This is needed because when making the sensor loss probability maps, we were running into numerican stability issues, giving weird results. Adding this small numbers, makes the behaviour consistent
noBackgroundFlux = 0
station = "196"

hitmaps = Helper20240606.loadHitmaps(station, noBackgroundFlux, backgroundFlux)

#test_hitmap = hitmaps["196-physics"]

print("Validating the hitmap files:")
for map in hitmaps:
    print("  - Map {}".format(map))
    hitmaps[map].validate()

## Create Fluxmap Histograms

In [ ]:
hitmap_histograms = {}

for angle_dir, angle, beta in Helper20240606.angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"

    hitmap_histograms[hitmap_key] = {
        "physics": hitmaps[hitmap_key].getHisto(
            f"station{station}-{angle_dir}-{angle}urad-{int(beta*100)}cm",
            f"Station {station} - {angle_dir} crossing angle, {angle} urad; {int(beta*100)}cm Beta star",
                                                ),
        "background": hitmaps[hitmap_key+"-background"].getHisto(
            f"station{station}-background-{angle_dir}-{angle}urad-{int(beta*100)}cm",
            f"Station {station} with {backgroundTitle} Background - {angle_dir} crossing angle, {angle} urad; {int(beta*100)}cm Beta star",
                                                ),
    }


## Needed Shifts

In [ ]:
needed_shifts = Helper20240606.getNeededShifts(station)
for key in needed_shifts:
    print(f"{key} - {needed_shifts[key][0]} shifts over {needed_shifts[key][1]} mm")

### Compute positions with shifts

In [ ]:
sensor = new_sensor_ti
xSensorSize = sensor.maxX - sensor.minX
ySensorSize = sensor.maxY - sensor.minY

#### Nominal Positions

In [ ]:
nominal_positions = Helper20240606.getNominalPositions(hitmaps, xSensorSize)
for key in nominal_positions:
    print(f"{key}: {nominal_positions[key]}")

#### Adjusted Positions

In [ ]:
adjusted_positions = Helper20240606.getAdjustedPositions(nominal_positions, station)
for key in adjusted_positions:
    print(f"{key}: {adjusted_positions[key]}")

#### Shifts over Year

In [ ]:
yOffsets = {}
for angle_dir in ["vertical", "horizontal"]:
    yPositions = []
    for angle in [125, 250]:
        position_key = f"{angle_dir}-{angle}urad"

        _, startPositionY = adjusted_positions[position_key]
        yPositions += [startPositionY]

        yCenters = Helper20240606.computeYCenters196(startPositionY, angle_dir, ySensorSize)
        yOffsets[position_key] = Helper20240606.computeShifts(yCenters, needed_shifts[position_key])

    positionY = max(yPositions)
    yCenters = Helper20240606.computeYCenters196(positionY, angle_dir, ySensorSize)
    yOffsets[angle_dir] = Helper20240606.computeShifts(yCenters, needed_shifts[angle_dir])

print(yOffsets)

#### Positions to use

In [ ]:
shift_str = "{angle_dir}"
#shift_str = "{angle_dir}-{angle}urad"

#### Sensor Positions on Hitmap with shifts

In [ ]:
c = []
p = []

for angle_dir, angle, beta in Helper20240606.angle_beta:
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
    c1, p1 = Helper20240606.plotShiftsOnFluxmap(
        sensor = sensor,
        positions = yOffsets[shift_str.format(angle_dir = angle_dir, angle = angle, beta = beta)],
        hitmap = hitmaps[hitmap_key],
        histogram = hitmap_histograms[hitmap_key]["physics"],
        base_name = hitmap_key,
        title = f"{angle_dir} ${angle} \\mu\\text{{rad}}$ - $\\beta^* = {int(beta*100)} \\text{{cm}}$",
    )
    c.append(c1)
    p.append(p1)

for canvas in c:
    canvas.Draw()

## Plot Pixel Occupancy in positions

In [ ]:
def plotOccupancy(angle_dir, angle, beta, minV, maxV):
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"
    return Helper20240606.plotOccupancy(
        sensor = sensor,
        positions = yOffsets[shift_str.format(angle_dir = angle_dir, angle = angle, beta = beta)],
        hitmap = hitmaps[hitmap_key + "-background"],
        name = f"_{angle_dir}-{angle}urad-{int(beta*100)}cm",
        minV = minV,
        maxV = maxV,
    )


### Vertical crossing angle, 250 urad; 50 cm $\beta^*$

In [ ]:
c, p = plotOccupancy("vertical", 250, 0.50, minV = 0.0447, maxV = 0.061)

c[0].Draw()
c[1].Draw()

### Vertical crossing angle, 250 urad; 20 cm $\beta^*$

In [ ]:
c, p = plotOccupancy("vertical", 250, 0.20, minV = 0.0447, maxV = 0.058)

c[0].Draw()
c[1].Draw()

### Vertical crossing angle, 125 urad; 50 cm $\beta^*$

In [ ]:
c, p = plotOccupancy("vertical", 125, 0.50, minV = 0.0447, maxV = 0.064)

c[0].Draw()
c[1].Draw()

### Vertical crossing angle, 125 urad; 20 cm $\beta^*$

In [ ]:
c, p = plotOccupancy("vertical", 125, 0.20, minV = 0.0447, maxV = 0.061)

c[0].Draw()
c[1].Draw()

### Horizontal crossing angle, 250 urad; 50 cm $\beta^*$

In [ ]:
c, p = plotOccupancy("horizontal", 250, 0.50, minV = 0.0447, maxV = 0.0501)

c[0].Draw()
c[1].Draw()

### Horizontal crossing angle, 250 urad; 15 cm $\beta^*$

In [ ]:
c, p = plotOccupancy("horizontal", 250, 0.15, minV = 0.0447, maxV = 0.0494)

c[0].Draw()
c[1].Draw()

### Horizontal crossing angle, 125 urad; 50 cm $\beta^*$

In [ ]:
c, p = plotOccupancy("horizontal", 125, 0.50, minV = 0.0447, maxV = 0.0532)

c[0].Draw()
c[1].Draw()

### Horizontal crossing angle, 125 urad; 15 cm $\beta^*$

In [ ]:
c, p = plotOccupancy("horizontal", 125, 0.15, minV = 0.0447, maxV = 0.0523)

c[0].Draw()
c[1].Draw()

In [ ]:
print(p[0]['histograms']['pos_0'].GetMaximum())
print(p[1]['histograms']['pos_0'].GetMaximum())
print(p[0]['histograms']['pos_0'].GetMinimum())

## Study Toys

### Useful Functions

In [ ]:
def calculateSensorEventLosses(sensor):
    min_occ = None
    max_occ = None
    for pad_idx in range(256):
        for epoch in range(len(sensor.padVec[pad_idx].doses)):
            occ = sensor.padVec[pad_idx].doses[epoch]['occupancy']

            if min_occ is None or occ < min_occ:
                min_occ = occ

            if max_occ is None or occ > max_occ:
                max_occ = occ

    print(f"The maximum occupancy is {max_occ} and the minimum occupancy is {min_occ}")

    def poisson_prob(occupancy, N):
        return ((occupancy**N) * exp(-occupancy))/(factorial(N))

    p0_min = poisson_prob(min_occ, 0)
    p1_min = poisson_prob(min_occ, 1)
    p2_min = 1 - p0_min - p1_min

    print("For the minimum occupancy:")
    print(f" - The probability of having 0 hits is: {p0_min}")
    print(f" - The probability of having 1 hits is: {p1_min}")
    print(f" - The probability of having 2 or more hits is: {p2_min}")

    p0_max = poisson_prob(max_occ, 0)
    p1_max = poisson_prob(max_occ, 1)
    p2_max = 1 - p0_max - p1_max

    print("For the maximum occupancy:")
    print(f" - The probability of having 0 hits is: {p0_max}")
    print(f" - The probability of having 1 hits is: {p1_max}")
    print(f" - The probability of having 2 or more hits is: {p2_max}")

    loss_min = 1-(p0_min + p1_min)**256
    loss_max = 1-(p0_max + p1_max)**256

    print(f"The event loss probability for the whole sensor should be between {loss_min} and {loss_max}")
    return loss_min, loss_max

In [ ]:
def getDataCat(df: pandas.DataFrame, category = None):
    category_ext = ""
    if category is not None:
        category_ext = f"_{category}"

    num_toys = len(df)

    event_loss_counts = df['event_loss'+category_ext].value_counts()
    mean_active_pads = df['active_pads'+category_ext].mean()
    std_active_pads = df['active_pads'+category_ext].std()
    mean_occupancy = df['sensor_occupancy'+category_ext].mean()
    std_occupancy = df['sensor_occupancy'+category_ext].std()
    mean_bit_length = df['bit_length'+category_ext].mean()
    std_bit_length = df['bit_length'+category_ext].std()

    data = [event_loss_counts[True], float(event_loss_counts[True])/num_toys, mean_active_pads, std_active_pads, mean_occupancy, std_occupancy, mean_bit_length, std_bit_length]

    return data

In [ ]:
def extractInfoFromToyCache(toy_cache):
    columns = ["event_loss_count", "event_loss_fraction", "active_pads_mean", "active_pads_std", "occupancy_mean", "occupancy_std", "bit_length_mean", "bit_length_std"]
    data = []
    left_data = []
    right_data = []
    for shift_index in range(len(toy_cache)):
        toy_df = toy_cache[shift_index]

        data += [getDataCat(toy_df)]
        left_data += [getDataCat(toy_df, "left")]
        right_data += [getDataCat(toy_df, "right")]

    info_df = pandas.DataFrame(data, columns=columns)
    left_info_df = pandas.DataFrame(left_data, columns=columns)
    right_info_df = pandas.DataFrame(right_data, columns=columns)

    return info_df, left_info_df, right_info_df

### Throw Toys

In [ ]:
def studyToys(angle_dir, angle, beta, num_toys = 1000):
    hitmap_key = f"{angle_dir}-{angle}urad-{int(beta*100)}cm"

    base_positions = yOffsets[shift_str.format(angle_dir = angle_dir, angle = angle, beta = beta)]
    hitmap = hitmaps[hitmap_key + "-background"]

    edge = hitmap.detectorEdge * 1000 # Convert to mm for drawing
    xSensorSize = sensor.maxX - sensor.minX
    offsetX = edge + xSensorSize/2

    center_pos = ceil(len(base_positions[0])/2)

    full_shift_positions = [(offsetX, base_positions[0][center_pos] + step*1.3) for step in [-1, 0, 1]]
    coverage_positions = [(offsetX, base_positions[0][center_pos] + step*1.3) for step in [0, 1/4, 2/4, 3/4]]

    positions = full_shift_positions + coverage_positions

    for sensor_idx in range(len(base_positions)):
        positions += [(offsetX, i) for i in base_positions[sensor_idx]]
    print(f"There are {len(positions)} positions: {positions}")

    sensor.setShifts(positions)
    sensor.calculateFlux(hitmap)

    toy_cache = sensor.simulateToys(numToys=num_toys)

    calculateSensorEventLosses(sensor)

    info_df, left_info_df, right_info_df = extractInfoFromToyCache(toy_cache)

    print(f"Maximum bit length: {info_df['bit_length_mean'].max()}; left - {left_info_df['bit_length_mean'].max()}; right - {right_info_df['bit_length_mean'].max()}")
    print(info_df)
    print(left_info_df)
    print(right_info_df)

    fig = sensor.plotToyActivePads(toyCache=toy_cache)

    fig = sensor.plotToySensorOccupancy(toyCache=toy_cache)

    fig = sensor.plotToyEventSize(toyCache=toy_cache)

    return

#### Vertical crossing angle, 250 urad; 50 cm $\beta^*$

In [ ]:
studyToys("vertical", 250, 0.50, num_toys = 10000)

#### Vertical crossing angle, 250 urad; 20 cm $\beta^*$

In [ ]:
studyToys("vertical", 250, 0.20, num_toys = 10000)

#### Vertical crossing angle, 125 urad; 50 cm $\beta^*$

In [ ]:
studyToys("vertical", 125, 0.50, num_toys = 10000)

#### Vertical crossing angle, 125 urad; 20 cm $\beta^*$

In [ ]:
studyToys("vertical", 125, 0.20, num_toys = 10000)

#### Horizontal crossing angle, 250 urad; 50 cm $\beta^*$

In [ ]:
studyToys("horizontal", 250, 0.50, num_toys = 10000)

#### Horizontal crossing angle, 250 urad; 15 cm $\beta^*$

In [ ]:
studyToys("horizontal", 250, 0.15, num_toys = 10000)

#### Horizontal crossing angle, 125 urad; 50 cm $\beta^*$

In [ ]:
studyToys("horizontal", 125, 0.50, num_toys = 10000)

#### Horizontal crossing angle, 125 urad; 15 cm $\beta^*$

In [ ]:
studyToys("horizontal", 125, 0.15, num_toys = 10000)